In [1]:
import sys
import os

# 获取当前工作目录（用于替代 __file__）
current_dir = os.getcwd()

target_path1 = os.path.abspath(os.path.join(current_dir, '../3_move'))
target_path2 = os.path.abspath(os.path.join(current_dir, '../1_getPointCloud'))

# 将路径添加到 sys.path
if target_path1 not in sys.path:
    sys.path.append(target_path1)
    
if target_path2 not in sys.path:
    sys.path.append(target_path2)
    
# 检查路径是否添加成功
print("Current Python Path:")
print("\n".join(sys.path))


Current Python Path:

/opt/ros_ws/devel/lib/python3/dist-packages
/opt/ros/noetic/lib/python3/dist-packages
/usr/lib/python38.zip
/usr/lib/python3.8
/usr/lib/python3.8/lib-dynload
/usr/local/lib/python3.8/dist-packages
/usr/lib/python3/dist-packages
/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode/3_move
/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode/1_getPointCloud


In [2]:
from math import atan2, sqrt
def matrix_to_rpy_and_translation(matrix):
    """
    从4x4变换矩阵中提取RPY（Roll, Pitch, Yaw）和位移信息，并返回一个列表形式。
    
    参数:
        matrix: 4x4变换矩阵 (numpy.ndarray)
        
    返回:
        result: 一个包含 [roll, pitch, yaw, x, y, z] 的列表
    """
    if matrix.shape != (4, 4):
        raise ValueError("输入必须是一个 4x4 矩阵")
    
    # 提取旋转矩阵
    rotation = matrix[:3, :3]
    
    # 提取位移
    translation = matrix[:3, 3]
    
    # 计算RPY角
    pitch = atan2(-rotation[2, 0], sqrt(rotation[0, 0]**2 + rotation[1, 0]**2))
    roll = atan2(rotation[2, 1], rotation[2, 2])
    yaw = atan2(rotation[1, 0], rotation[0, 0])

    # 返回结果作为列表
    return [roll, pitch, yaw], list(translation)


In [3]:
from save_point_cloud import PointCloudSaver
import rospy
point_cloud_saver = PointCloudSaver()

# 等待数据准备
rospy.loginfo("Waiting for data...")
rospy.sleep(1)  # 等待话题数据发布

# 保存点云
original_file = "/opt/ros_ws/zed_point_cloud3.ply"
world_file = "/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode/2_perception/mesh/zed_point_cloud_world3.ply"
point_cloud_saver.save_point_clouds(original_file, world_file)


from PickAndPlace import PickAndPlace
pick_place = PickAndPlace(approach_distance=0.1)



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[INFO] [1733105713.001948, 0.000000]: Waiting for data...
[INFO] [1733105718.487176, 4843.031000]: Original point cloud saved to /opt/ros_ws/zed_point_cloud3.ply
[INFO] [1733105718.488080, 4843.032000]: Requesting transform from world to left_camera_link_optical...
[INFO] [1733105718.488780, 4843.032000]: Transform found: header: 
  seq: 0
  stamp: 
    secs: 4843
    nsecs:         0
  frame_id: "world"
child_frame_id: "left_camera_link_optical"
transform: 
  translation: 
    x: 0.1617059471314676
    y: -0.08595541418983325
    z: 0.5375340392228071
  rotation: 
    x: 0.6680705450980908
    y: 0.6743383043649714
    z: 0.2205368687625264
    w: 0.2243057902840126
[INFO] [1733105718.552351, 4843.096000]: Transformed point cloud saved to /opt/ros_ws/src/franka_zed_gazebo/scripts/mycode/2_perception/mesh/zed_point_clou

[ WARN] [1733105718.656389881]: Link zed2_holder has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.


[INFO] [1733105719.756653, 4844.285000]: MoveRobot initialized successfully.
[INFO] [1733105719.768327, 4844.311000]: Waiting for gripper action servers...
[INFO] [1733105720.083408, 4844.626000]: Gripper action servers ready.


In [4]:
import open3d as o3d
import numpy as np

def filter_point_cloud_by_depth(ply_path, depth_threshold=0.01):
    """
    读取点云文件并过滤掉深度低于指定阈值的点。

    Args:
        ply_path (str): 点云文件的路径。
        depth_threshold (float): 深度阈值，低于该值的点将被过滤。

    Returns:
        o3d.geometry.PointCloud: 过滤后的点云对象。
    """
    # 读取点云文件
    point_cloud = o3d.io.read_point_cloud(ply_path)
    if not point_cloud.has_points():
        raise ValueError(f"Failed to read point cloud from {ply_path}")

    # 获取点云的点坐标
    points = np.asarray(point_cloud.points)

    # 过滤深度小于阈值的点
    filtered_points = points[points[:, 2] >= depth_threshold]

    # 创建新的点云对象
    filtered_point_cloud = o3d.geometry.PointCloud()
    filtered_point_cloud.points = o3d.utility.Vector3dVector(filtered_points)

    # 保留颜色信息（如果存在）
    if point_cloud.has_colors():
        colors = np.asarray(point_cloud.colors)
        filtered_colors = colors[points[:, 2] >= depth_threshold]
        filtered_point_cloud.colors = o3d.utility.Vector3dVector(filtered_colors)

    return filtered_point_cloud
zed_ply_path = "mesh/zed_point_cloud_world3.ply"
filtered_point_cloud = filter_point_cloud_by_depth(zed_ply_path, depth_threshold=0.01)
o3d.visualization.draw_geometries([filtered_point_cloud], window_name="Filtered Point Cloud")

In [5]:
def calculate_max_layer(filtered_point_cloud, layer_height=0.04):
    """
    计算点云的最高层数 (MaxLayer)，基于 z 轴最大值，使用四舍五入计算。

    Args:
        filtered_point_cloud (o3d.geometry.PointCloud): 已过滤的点云对象。
        layer_height (float): 每一层的高度，默认值为 0.04。

    Returns:
        int: 最高层数 (MaxLayer)。
    """
    # 提取点云的 z 轴最大值
    points = np.asarray(filtered_point_cloud.points)
    z_max = np.max(points[:, 2])

    # 计算最高层数，严格四舍五入
    max_layer = int(np.floor(z_max / layer_height + 0.5))

    return max_layer

# 示例调用
max_layer = calculate_max_layer(filtered_point_cloud, layer_height=0.04)
print(f"MaxLayer: {max_layer}")


MaxLayer: 1


In [6]:
import copy

def register_and_filter(pointcloud, mesh, voxel_size=0.01):
    # 确保输入的 mesh 是 TriangleMesh 对象
    if isinstance(mesh, o3d.geometry.TriangleMesh):
        # 转换网格为点云
        mesh_pointcloud = mesh.sample_points_uniformly(number_of_points=1000)
    elif isinstance(mesh, o3d.geometry.PointCloud):
        # 如果已经是点云，直接使用
        mesh_pointcloud = copy.deepcopy(mesh)
    else:
        raise TypeError("Input mesh must be of type o3d.geometry.TriangleMesh or o3d.geometry.PointCloud")

    # 下采样点云并计算法线和特征
    def preprocess_point_cloud(pcd, voxel_size):
        pcd_down = pcd.voxel_down_sample(voxel_size)
        pcd_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
        pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
            pcd_down,
            search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*5, max_nn=100))
        return pcd_down, pcd_fpfh

    # 粗配准
    def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
        distance_threshold = voxel_size * 1.5
        result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source_down, target_down, source_fpfh, target_fpfh, True,
            distance_threshold,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
            4,
            [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
             o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
            o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 200))
        return result

    # 精配准
    def refine_registration(source, target, initial_transformation, voxel_size):
        distance_threshold = voxel_size * 1
        result = o3d.pipelines.registration.registration_icp(
            source, target, distance_threshold, initial_transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000000)
        )
        return result

    # 下采样点云并计算特征
    source_down, source_fpfh = preprocess_point_cloud(mesh_pointcloud, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(pointcloud, voxel_size)

    # 粗配准
    coarse_result = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)

    # 精配准
    refined_result = refine_registration(mesh_pointcloud, pointcloud, coarse_result.transformation, voxel_size)
    print(refined_result)

    # 将网格点云变换到目标点云的空间
    transform = refined_result.transformation
    transformed_mesh_pointcloud = mesh_pointcloud.transform(transform)

    # 创建包围盒裁剪目标点云
    bounding_box = transformed_mesh_pointcloud.get_axis_aligned_bounding_box()

    # 找到位于包围盒内部的点索引
    indices_inside_box = bounding_box.get_point_indices_within_bounding_box(pointcloud.points)

    # 获取位于包围盒外部的点
    indices_outside_box = [i for i in range(len(pointcloud.points)) if i not in indices_inside_box]

    # 创建新的点云
    remaining_pointcloud = pointcloud.select_by_index(indices_outside_box)
    deleted_pointcloud = pointcloud.select_by_index(indices_inside_box)

    return transform, remaining_pointcloud, deleted_pointcloud, refined_result.fitness


def filter_point_cloud_by_z(pcd, z_threshold=-0.001):
    """
    过滤点云中所有 z < z_threshold 的点。
    
    Args:
        pcd (open3d.geometry.PointCloud): 输入的点云对象。
        z_threshold (float): z 轴的过滤阈值，小于该值的点将被移除。
        
    Returns:
        open3d.geometry.PointCloud: 过滤后的点云对象。
    """
    # 转换点云为 numpy 数组
    points = np.asarray(pcd.points)
    
    # 过滤 z 值小于阈值的点
    filtered_points = points[points[:, 2] >= z_threshold]
    
    # 创建新的点云对象
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
    
    # 保留原始颜色（如果存在）
    if pcd.has_colors():
        colors = np.asarray(pcd.colors)
        filtered_colors = colors[points[:, 2] >= z_threshold]
        filtered_pcd.colors = o3d.utility.Vector3dVector(filtered_colors)
    
    return filtered_pcd

In [7]:
# 读取cube
# 定义文件路径
cube_obj_path = "mesh/cube_0.obj"
zed_ply_path = "mesh/zed_point_cloud_world3.ply"

# 读取 cube_0.obj 点云
cube_mesh = o3d.io.read_triangle_mesh(cube_obj_path)
cube_mesh.compute_vertex_normals()  # 计算法线以便更好显示
cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=10000)  # 转为点云
cube_point_cloud

coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1,  # 坐标轴大小，可以根据需要调整
    origin=[0, 0, 0]  # 坐标轴的原点
)


In [ ]:
def pointcloud_splite_by_layers(point_cloud, max_layer, layer_thickness=0.04, slice_tolerance=0.005):
    """
    将点云切割成多层，每层0.04个高度，然后依次取每个方块放到固定位置并显示。
    """
    # 获取点和颜色信息直接使用 open3d 的方法
    points = point_cloud.points
    colors = point_cloud.colors if point_cloud.has_colors() else o3d.utility.Vector3dVector([[1, 1, 1]] * len(points))
    movecount = 0

    for layer in range(max_layer-1, -1, -1):
        z_base = layer * layer_thickness
        z_min = z_base + slice_tolerance
        z_max = z_base + layer_thickness + slice_tolerance

        # 筛选属于该层的点
        mask = [z_min <= pt[2] <= z_max for pt in points]
        layer_points = [points[i] for i in range(len(points)) if mask[i]]
        layer_colors = [colors[i] for i in range(len(colors)) if mask[i]]

        if len(layer_points) > 0:
            print(f"Processing layer {layer} (Z range: {z_min:.4f} to {z_max:.4f})")
            
            # 创建一个新的点云对象用于显示
            layer_point_cloud = o3d.geometry.PointCloud()
            layer_point_cloud.points = o3d.utility.Vector3dVector(layer_points)
            layer_point_cloud.colors = o3d.utility.Vector3dVector(layer_colors)
            
            remaining_pointcloud_count = 10000
            countdown = 100
            # 使用 open3d 可视化点云
            # o3d.visualization.draw_geometries([layer_point_cloud], window_name=f"Layer {layer} (Z range: {z_min:.4f} to {z_max:.4f})")
            while remaining_pointcloud_count > 200 and countdown>0:
                countdown = countdown - 1
                
                
                transform, remaining_pointcloud, deleted_pointcloud, fitness = register_and_filter(layer_point_cloud, cube_point_cloud)
                remaining_pointcloud_count = len(remaining_pointcloud.points)
                
                if fitness>0.3:
                    movecount = movecount+1
                    layer_point_cloud = remaining_pointcloud
                    # 如果匹配度高，则移动将其移动到指定位置6
                    pick_rpy, pick_pos = matrix_to_rpy_and_translation(transform)
                    pick_rpy = [0, np.pi, pick_rpy[2]]
                    pick_pos_ = [a + b for a, b in zip(pick_pos, [0.00, 0.025, 0.11])]
                    place_rpy = [0, np.pi, np.pi/4]
                    pick_rpy = place_rpy
                    place_pos = [0.6 - (movecount%6)*0.1, 0.5-0.05*(movecount//6), 0.13]
                    print(place_pos)
                    
                    pick_place.pick_and_place(
                        pick_pos=pick_pos_,
                        pick_rpy=pick_rpy,
                        place_pos=place_pos,
                        place_rpy=place_rpy
                    )
                    # return transform, remaining_pointcloud, deleted_pointcloud
                
            # 这里debug用，直接return
            # return transform, remaining_pointcloud, deleted_pointcloud
T, remaining_pointcloud, deleted_pointcloud = pointcloud_splite_by_layers(filtered_point_cloud, max_layer)


Processing layer 0 (Z range: 0.0050 to 0.0450)
RegistrationResult with fitness=5.968000e-01, inlier_rmse=3.585965e-03, and correspondence_set size of 5968
Access transformation to get result.
[0.5, 0.5, 0.13]
[INFO] [1733105725.527895, 4850.000000]: Sending open goal: width: 0.08
speed: 0.1
[INFO] [1733105725.562260, 4850.055000]: Gripper opened successfully.
[INFO] [1733105733.817644, 4858.310000]: Move successful to position: [0.3488175798390157, 0.22767960566028128, 0.22811562078522382] and RPY: [0, 3.141592653589793, 0.7853981633974483]
[INFO] [1733105733.818506, 4858.311000]: Generated waypoints:
[INFO] [1733105733.818930, 4858.312000]: Planning Cartesian path...
[INFO] [1733105733.823547, 4858.315000]: Path planning completed successfully!
[INFO] [1733105733.823996, 4858.317000]: Executing Cartesian path...
[INFO] [1733105734.942392, 4859.435000]: Grasp approach executed successfully.
[INFO] [1733105734.943458, 4859.435000]: Sending grasp goal: width: 0.05
epsilon: 
  inner: 0.01

In [8]:
T

array([[ 0.12018957, -0.99139992,  0.05177518,  0.54396382],
       [ 0.98544627,  0.12545789,  0.11469943,  0.1471949 ],
       [-0.12020861,  0.03723598,  0.99205009,  0.06128015],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [10]:
matrix_to_rpy_and_translation(T)

([0.03751676351455387, 0.12050001348623428, 1.4494311361457375],
 [0.5439638234072214, 0.1471949043982344, 0.06128015255172841])

[ WARN] [1733100922.529691242]: Link zed2_holder has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.


[INFO] [1733100923.748118, 55.421000]: MoveRobot initialized successfully.
[INFO] [1733100923.762388, 56.682000]: Waiting for gripper action servers...
[INFO] [1733100924.085310, 57.005000]: Gripper action servers ready.


In [12]:
pick_rpy, pick_pos = matrix_to_rpy_and_translation(T)
pick_rpy = [0, np.pi, pick_rpy[2]]
pick_pos

[0.5439638234072214, 0.1471949043982344, 0.06128015255172841]

In [38]:
pick_pos_ = [a + b for a, b in zip(pick_pos, [0.01, 0.02, 0.10])]
place_rpy = [0, np.pi, np.pi/4]
pick_rpy = place_rpy
place_pos = [0.6, 0.5, 0.13]


In [39]:
pick_pos_

[0.5539638234072214, 0.1671949043982344, 0.16128015255172842]

In [45]:
pick_pos_, place_pos = [0.6, 0.5, 0.13+0.1], [a + b for a, b in zip(pick_pos, [0.01, 0.02, 0.0])]

In [46]:
pick_place.pick_and_place(
    pick_pos=pick_pos_,
    pick_rpy=pick_rpy,
    place_pos=place_pos,
    place_rpy=place_rpy
)

[INFO] [1733101297.646593, 430.456000]: Sending open goal: width: 0.08
speed: 0.1
[INFO] [1733101297.681474, 430.491000]: Gripper opened successfully.
[INFO] [1733101303.926676, 436.734000]: Move successful to position: [0.6, 0.5, 0.33] and RPY: [0, 3.141592653589793, 0.7853981633974483]
[INFO] [1733101303.927569, 436.735000]: Generated waypoints:
[INFO] [1733101303.928005, 436.736000]: Waypoint 0: Position(0.6, 0.5, 0.33), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [1733101303.928388, 436.736000]: Waypoint 1: Position(0.6, 0.5, 0.329), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [1733101303.928732, 436.736000]: Waypoint 2: Position(0.6, 0.5, 0.328), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [1733101303.929088, 436.737000]: Waypoint 3: Position(0.6, 0.5, 0.327), Orientation(-0.3826834323650898, 0.

[WARN] [1733101306.094735, 438.902000]: Grasp failed.


[INFO] [1733101306.095674, 438.903000]: Generated waypoints:
[INFO] [1733101306.096151, 438.904000]: Waypoint 0: Position(0.6, 0.5, 0.23), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [1733101306.096493, 438.904000]: Waypoint 1: Position(0.6, 0.5, 0.231), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [1733101306.096837, 438.904000]: Waypoint 2: Position(0.6, 0.5, 0.232), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [1733101306.097161, 438.905000]: Waypoint 3: Position(0.6, 0.5, 0.23299999999999998), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [1733101306.097461, 438.905000]: Waypoint 4: Position(0.6, 0.5, 0.23399999999999999), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [173310130

[WARN] [1733101323.033034, 455.835000]: Failed to open gripper.


[INFO] [1733101323.033893, 455.835000]: Generated waypoints:
[INFO] [1733101323.034391, 455.836000]: Waypoint 0: Position(0.5539638234072214, 0.1671949043982344, 0.1112801525517284), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [1733101323.034744, 455.836000]: Waypoint 1: Position(0.5539638234072214, 0.1671949043982344, 0.11228015255172841), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [1733101323.035105, 455.837000]: Waypoint 2: Position(0.5539638234072214, 0.16719490439823437, 0.11328015255172841), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [1733101323.035409, 455.837000]: Waypoint 3: Position(0.5539638234072214, 0.1671949043982344, 0.11428015255172841), Orientation(-0.3826834323650898, 0.9238795325112867, 2.3432602026631493e-17, 5.657130561438501e-17)
[INFO] [1733101323.035705, 455.837000]: Waypoin